In [1]:
import json
import requests
import urllib.request
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.DataFrame()
index = 0
last_date = ''

# 수정 변수들
coinName = "QTCON"
to_date = '2021-06-01'
num = 30
scale = '1'
if num == 0:
    type = 'days'
else:
    type = 'minutes/{}'.format(num)


def get_coin_data_url(coinName, type, scale, cnt=400) :
    addr = 'https://crix-api-endpoint.upbit.com/v1/crix/candles/'
    if type == 'minutes' :
        basic_url = addr + type + '/' + scale + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)
    else :
        basic_url = addr + type + '/' + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)

    return basic_url

basic_url = get_coin_data_url(coinName,type,scale)
url = basic_url

def get_upbit_data(url, last_date, to_date) :
    global df
    global index

    fail2GetData = False
    response = ''
    while True:
        try :
            response = requests.get(url)
        except Exception as e:
            time.sleep(20)
            continue
        if str(response) == '<Response [200]>':
            break
        time.sleep(10)
    if ( fail2GetData )  :
        exit()

    data = response.json()

    if index == 0:
        df = pd.DataFrame(data)
        date = ''
        date = data[len(data)-1]['candleDateTime']
        index +=1

    else:
        df_for = pd.DataFrame(data)
        df = df.append(df_for,ignore_index=True)
        date = ''
        date = data[len(data)-1]['candleDateTime']


    return date




In [2]:

while (1) :
    last_date = get_upbit_data(url, last_date, to_date)
    tmp1 = last_date.split('T')
    if tmp1[0]  < to_date :
        break
    tmp2 = tmp1[1].split('+')
    target_date = tmp1[0] + ' ' + tmp2[0]
    url = basic_url + '&to=' + target_date    #to=2019-11-27 04:01:00
    time.sleep(2)

idx_todate = df.index[(df.candleDateTimeKst == '{}T00:00:00+09:00'.format(to_date))].tolist()[0]+1
idx_todate

df = df.drop(df.index[idx_todate:])
df

,code,candleDateTime,candleDateTimeKst,openingPrice,highPrice,lowPrice,tradePrice,candleAccTradeVolume,candleAccTradePrice,timestamp,unit
0,CRIX.UPBIT.KRW-QTCON,2021-06-12T08:30:00+00:00,2021-06-12T17:30:00+09:00,23.3,23.3,23.2,23.2,5.035514e+05,1.168697e+07,1623486762789,30
1,CRIX.UPBIT.KRW-QTCON,2021-06-12T08:00:00+00:00,2021-06-12T17:00:00+09:00,23.6,23.7,23.0,23.2,1.245662e+07,2.899820e+08,1623486570740,30
2,CRIX.UPBIT.KRW-QTCON,2021-06-12T07:30:00+00:00,2021-06-12T16:30:00+09:00,24.1,24.2,23.5,23.6,6.990977e+06,1.668503e+08,1623484796015,30
3,CRIX.UPBIT.KRW-QTCON,2021-06-12T07:00:00+00:00,2021-06-12T16:00:00+09:00,24.4,24.8,24.1,24.2,5.337817e+06,1.301828e+08,1623482993449,30
4,CRIX.UPBIT.KRW-QTCON,2021-06-12T06:30:00+00:00,2021-06-12T15:30:00+09:00,24.2,24.8,23.8,24.3,1.150076e+07,2.804744e+08,1623481199456,30
...,...,...,...,...,...,...,...,...,...,...,...
556,CRIX.UPBIT.KRW-QTCON,2021-05-31T17:00:00+00:00,2021-06-01T02:00:00+09:00,52.6,52.7,52.3,52.4,7.925233e+05,4.158999e+07,1622482168004,30
557,CRIX.UPBIT.KRW-QTCON,2021-05-31T16:30:00+00:00,2021-06-01T01:30:00+09:00,52.4,52.7,52.3,52.6,6.720946e+05,3.527668e+07,1622480397914,30
558,CRIX.UPBIT.KRW-QTCON,2021-05-31T16:00:00+00:00,2021-06-01T01:00:00+09:00,52.4,52.8,52.2,52.4,1.788808e+06,9.389585e+07,1622478547916,30
559,CRIX.UPBIT.KRW-QTCON,2021-05-31T15:30:00+00:00,2021-06-01T00:30:00+09:00,52.5,52.7,52.2,52.5,4.414198e+05,2.315340e+07,1622476737626,30


In [3]:
df['Date'] = 0
df['hour'] = 0
df['minute'] = 0

In [4]:
df['Date'][0] = df['candleDateTimeKst'][0].split('T')
df['Date'][0][0]


'2021-06-12'

In [5]:
for i in range(len(df)):
    df['Date'] [i] = df['candleDateTimeKst'][i].split('T')
    df['hour'] [i] = df['Date'][i][1].split('+')[0]
    df['minute'] [i] = df['hour'][i].split(':')[1]
    df['hour'] [i] = df['hour'][i].split(':')[0]
    df['Date'] [i] = df['Date'][i][0]


In [6]:
df

,code,candleDateTime,candleDateTimeKst,openingPrice,highPrice,lowPrice,tradePrice,candleAccTradeVolume,candleAccTradePrice,timestamp,unit,Date,hour,minute
0,CRIX.UPBIT.KRW-QTCON,2021-06-12T08:30:00+00:00,2021-06-12T17:30:00+09:00,23.3,23.3,23.2,23.2,5.035514e+05,1.168697e+07,1623486762789,30,2021-06-12,17,30
1,CRIX.UPBIT.KRW-QTCON,2021-06-12T08:00:00+00:00,2021-06-12T17:00:00+09:00,23.6,23.7,23.0,23.2,1.245662e+07,2.899820e+08,1623486570740,30,2021-06-12,17,0
2,CRIX.UPBIT.KRW-QTCON,2021-06-12T07:30:00+00:00,2021-06-12T16:30:00+09:00,24.1,24.2,23.5,23.6,6.990977e+06,1.668503e+08,1623484796015,30,2021-06-12,16,30
3,CRIX.UPBIT.KRW-QTCON,2021-06-12T07:00:00+00:00,2021-06-12T16:00:00+09:00,24.4,24.8,24.1,24.2,5.337817e+06,1.301828e+08,1623482993449,30,2021-06-12,16,0
4,CRIX.UPBIT.KRW-QTCON,2021-06-12T06:30:00+00:00,2021-06-12T15:30:00+09:00,24.2,24.8,23.8,24.3,1.150076e+07,2.804744e+08,1623481199456,30,2021-06-12,15,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,CRIX.UPBIT.KRW-QTCON,2021-05-31T17:00:00+00:00,2021-06-01T02:00:00+09:00,52.6,52.7,52.3,52.4,7.925233e+05,4.158999e+07,1622482168004,30,2021-06-01,02,0
557,CRIX.UPBIT.KRW-QTCON,2021-05-31T16:30:00+00:00,2021-06-01T01:30:00+09:00,52.4,52.7,52.3,52.6,6.720946e+05,3.527668e+07,1622480397914,30,2021-06-01,01,30
558,CRIX.UPBIT.KRW-QTCON,2021-05-31T16:00:00+00:00,2021-06-01T01:00:00+09:00,52.4,52.8,52.2,52.4,1.788808e+06,9.389585e+07,1622478547916,30,2021-06-01,01,0
559,CRIX.UPBIT.KRW-QTCON,2021-05-31T15:30:00+00:00,2021-06-01T00:30:00+09:00,52.5,52.7,52.2,52.5,4.414198e+05,2.315340e+07,1622476737626,30,2021-06-01,00,30


In [7]:

drops= ['Date','unit','candleAccTradePrice','openingPrice','code','candleDateTime','timestamp','candleDateTimeKst']
df=df.drop(drops,axis=1)
df=df[['hour','minute','tradePrice','highPrice','lowPrice','candleAccTradeVolume']]
df.head()

,hour,minute,tradePrice,highPrice,lowPrice,candleAccTradeVolume
0,17,30,23.2,23.3,23.2,5.035514e+05
1,17,0,23.2,23.7,23.0,1.245662e+07
2,16,30,23.6,24.2,23.5,6.990977e+06
3,16,0,24.2,24.8,24.1,5.337817e+06
4,15,30,24.3,24.8,23.8,1.150076e+07


In [8]:
df['tradePrice'][0]

23.2

In [9]:
# df['rsi_ud'] = 0

# for j in range(2):
#     upper = 0
#     down = 0
#     for i in range(14):
#         if df['tradePrice'][i+j] > df['tradePrice'][i+j+1]:
#             upper += (df['tradePrice'][i+j] - df['tradePrice'][i+j+1])
#             print(upper)
#         elif df['tradePrice'][i+j] < df['tradePrice'][i+j+1]:
#             down += df['tradePrice'][i+j+1] - df['tradePrice'][i+j]
#             print(down)
#         else:
#             continue
#     print(upper,down)
#     AU = upper/14
#     AD = down/14
#     RS = AU / AD
#     # print(RS)
#     RSI = (RS/(1+RS))
#     # print(RSI)

In [10]:
def EMA_RSI(n,m,list): # n 참고데이터수 m 알고싶은 데이터수
    a = 1/(n+1)
    upper = 0
    down = 0
    for j in range(m):
        for i in range(n):
            if df['tradePrice'][i+j] > df['tradePrice'][i+j+1]:
                upper += a*((1-a)**i)*(df['tradePrice'][i+j] - df['tradePrice'][i+j+1])
                # print(upper)
            elif df['tradePrice'][i+j] < df['tradePrice'][i+j+1]:
                down += a*((1-a)**i)*(df['tradePrice'][i+j+1] - df['tradePrice'][i+j])
                # print(down)
            else:
                continue
        print(upper,down)
        rs = upper/down
        print(rs)
        thx = (rs/(1+rs))*100
        print(thx , j)
    
            

In [11]:
EMA_RSI(14,30,df['tradePrice'])


0.07439397382148746 0.1325686020285235
0.5611734051889665
35.94561650382722 0
0.15410180291593834 0.3126704291834569
0.49285697825143654
33.01434668100044 1
0.24765962821725637 0.477065243992314
0.5191315681367189
34.17291688391786 2
0.3479001553258114 0.6484094419833194
0.536543937826805
34.91888026226314 3
0.4553007200849775 0.9825435310807769
0.46338987096496015
31.665510344102664 4
0.5632298966126553 1.3677329403046454
0.41179815153622235
29.168344716143146 5
0.6360111571780247 1.834813077711975
0.3466353956726399
25.7408498830893 6
0.713991079212349 2.3318458075818946
0.3061913771874788
23.44154023178266 7
0.7689695671062676 2.8643808752996653
0.26845925894049255
21.164200351592594 8
0.827875089849752 3.427119890449858
0.24156583846300214
19.45654680399783 9
0.8909881499320566 4.052819151820746
0.21984404350531567
18.022307415100123 10
0.9586092857345258 4.739524377261223
0.20225854103285923
16.82321515130174 11
0.9882033598085999 5.475279975947449
0.180484534882182
15.28902154573